In [2]:
from src.inference import simpleTryDiffusion
from diffusers import DDPMScheduler,AutoencoderKL,StableDiffusionPipeline,StableDiffusionImg2ImgPipeline,UNet2DConditionModel
from transformers import AutoModel
import torch
from PIL import Image
import random

/home/bh/anaconda3/envs/cvpr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-26 18:13:38,175] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
base_model_path="runwayml/stable-diffusion-v1-5"
vae_model_path = "./model/vae"
scheduler_path = "./model/scheduler"
image_encoder_path = "model/dinov2-G"
device = "cuda"

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
noise_scheduler = DDPMScheduler.from_pretrained(scheduler_path)
vae = AutoencoderKL.from_pretrained(vae_model_path)
image_encoder = AutoModel.from_pretrained(image_encoder_path)

In [ ]:
import os
os.environ['http_proxy']='http://127.0.0.1:7890'
os.environ['https_proxy']='http://127.0.0.1:7890'

In [ ]:
# load SD pipe
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)

In [ ]:
# 先获得pairs 列表
data_pair = list(open("dataset/VITON-HD/test_pairs.txt"))
# 处理文件内部
for i in range(len(data_pair)):
    flag = data_pair[i].find("jpg")
    path1 = data_pair[i][0:flag+3]
    path2 = data_pair[i][flag+4:-1]
    data_pair[i] = [path1, path2]

In [ ]:
# read image
random_int = random.randint(0, len(data_pair))
path1 = data_pair[random_int][0]
path2 = data_pair[random_int][1]
people_model = Image.open("dataset/VITON-HD/test/image/"+path1).convert("RGB")
cloth = Image.open("dataset/VITON-HD/test/cloth/"+path1).convert("RGB")
people_try = Image.open("dataset/VITON-HD/test/image/"+path2).convert("RGB")
mask = Image.open("dataset/VITON-HD/test/agnostic-v3.2/"+path2).convert("RGB")
image_grid([people_model.resize((256, 256)), 
            cloth.resize((256, 256)),
            people_try.resize((256, 256)),
            mask.resize((256, 256))],
            rows=1, cols=4)

In [ ]:
# load SimpleOnUnet
success_step = "85000"
model_path = "simpleOnUnet/"+str(success_step)+"/mp_rank_00_model_states.pt"
sd_model_dict = torch.load(model_path)["module"]

In [ ]:
# load ip-adapter
sd_model = simpleTryDiffusion(pipe,sd_model_dict,image_encoder,device)

In [ ]:
images = sd_model(cloth=cloth,mask=mask,num_samples=4)
grid = image_grid(images, 1, 4)
grid

In [1]:
from diffusers import UNet2DConditionModel
from model.attention_processor import *

/home/bh/anaconda3/envs/try_on/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-11-05 00:24:09,897] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
unet = UNet2DConditionModel.from_pretrained("./model/unet/")

In [3]:
attn_procs = {}
unet_sd = unet.state_dict()
for name in unet.attn_processors.keys():
    cross_attention_dim = None if name.endswith("attn1.processor") else unet.config.cross_attention_dim
    if name.startswith("mid_block"):
        hidden_size = unet.config.block_out_channels[-1]
    elif name.startswith("up_blocks"):
        block_id = int(name[len("up_blocks.")])
        hidden_size = list(reversed(unet.config.block_out_channels))[block_id]
    elif name.startswith("down_blocks"):
        block_id = int(name[len("down_blocks.")])
        hidden_size = unet.config.block_out_channels[block_id]
    if cross_attention_dim is None:
        attn_procs[name] = AttnProcessor()
    else:
        layer_name = name.split(".processor")[0]
        weights = {
            "to_k_ip.weight": unet_sd[layer_name + ".to_k.weight"],
            "to_v_ip.weight": unet_sd[layer_name + ".to_v.weight"],
            }
        attn_procs[name] = IPAttnProcessor(hidden_size=hidden_size, cross_attention_dim=cross_attention_dim)
        attn_procs[name].load_state_dict(weights)
unet.set_attn_processor(attn_procs)
adapter_modules = torch.nn.ModuleList(unet.attn_processors.values())